In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataset = pd.read_csv("train_df_clean.csv")

In [4]:
dataset.head()

,Unnamed: 0,num_of_siblings,PCR_01,PCR_02,PCR_03,PCR_06,PCR_10,risk,spread,covid,blood_A_AB
0,104,2.0,-0.573211,-0.719066,54.710912,-3.352526,7.0,Low,High,True,1
1,938,0.0,-0.269621,0.856951,261.491002,-2.242067,4.0,Low,High,True,0
2,152,5.0,-0.098708,-0.133210,152.614436,-2.529254,6.0,High,Low,False,0
3,2404,1.0,-0.626930,0.775240,283.656974,12.236025,3.0,High,High,True,1
4,2603,3.0,0.978812,-0.371731,317.362359,-6.819434,4.0,Low,High,False,0
